# NB.
Сначала все ячейки до первой модели, потом начиная с обязательности полей и по "one by one", а потом возврат к первой модели.

In [128]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold

USD = 24.5
EUR =  26.9

DETAILS = ['SqrTotal', 'NFloors',
          'NRooms', 'Floor',
          'SqrLive', 'SqrKitchen']
DETAILS_METROVKA = ["square", "floor_count"]

In [2]:
ADMIN_REGION = ['1000001',
 '1002241',
 '1002242',
 '1002243',
 '1002246',
 '1002247',
 '1002248',
 '1002249',
 '1002250',
 '1002253',
 '1003198',
 ]

In [3]:
MASS_REGION = ['1000000',
 '1000002',
 '1000003',
 '1000005',
 '1000007',
 '1000008',
 '1000009',
 '1000010',
 '1000011',
 '1000012',
 '1000013',
 '1000014',
 '1000015',
 '1000016',
 '1000017',
 '1002254',
 '1002255',
 '1002256',
 '1002307',
 '1002308',
 '1002379',
 '1002380',
 '1002381',
 '1002382',
 '1002396',
 '1002478',
 '1002497',
 '1002537',
 '1002558',
 '1002559',
 '1002773',
 '1002852',
 '1002853',
 '1002854',
 '1002855',
 '1003012',
 '1003023',
 '1003091',
 '1003092',
 '1003094',
 '1003199',
 '1003200',
 '1003201',
 '1003202',
 '1003203',
 '1003326',
 '1003327',
 '1003437',
 '1003438',
 '1003439',
 '1003441',
 '1003503',
 '1003504',
 '1003505',
 '1003506',
 '1003705',
 '1003706',
 '1003708',
 '1003709',
 '1003711',
 '1003712',
 '1003713',
 '1003737',
 '1003748',
 '1003753',
 '1003754',
 '201326594',
 '201326596',
 '201326597',
 '201326598',
 '201326599',
 '201326600',
 '201326601',
 '201326602',
 '201326603',
 '201326604',
 '201326605',
 '201326606',
 '201326607',
 '201326608',
 '201326610',
 '201326611',
 '201326612',
 '201326613',
 '201326614',
 '201326615',
 '201326616',
 '201326617',
 '201326618',
 '201326619',
 '201326621',
 '385875969',
 '385875970',
 '385875971',
 '385876140',
 '385876141',
 '385876142',
 '385876143',
 '385876144',
 '385876145',
 '385876146',
 '385876147',
 '385876148',
 '385876149',
 '385876158',
 '385876159',
 '419430401',
 '419430402']

In [4]:
BIULD_TYPE = [
 '5039',
 '5040',
 '5041',
 '5042',
 '5044',
 '5045',
 '5048',
 '5053',
 '5055',
 '5061',
 '5063',
 '5064',
 '5065',
 '5066',
 '5067',
 '5069',
 '5070',
 '20451',
 '20452',
 '20485',
 '20492',
 '21486',
 '21737',
 '21777',
 '21778',
 '16797715',
 '16797718',
 '201326911',
 '201326915',
 '201326916',
 '201326917',
 '201326918',
 '201326919',
 '201326920',
 '385876155']

In [5]:
ROOMS_TYPE = ['213', '10200', '10201', '10202']
CEIL_KIND = ['5109', '5110', '5111']
BIULD_KIND = [
 '5094',
 '5095',
 '5096',
 '5097',
 '5098',
 '5099',
 '5100',
 '5101',
 '20528',
 '20629',
 '20647',
 '20812',
 '20813',
 '21594',
 '16877220',
 '385875982',
 '385875985']

In [81]:
NOMINAL_DETAILS = [
    ('RegionCode', ADMIN_REGION),
    ('Massive', MASS_REGION),
    ('BldType', BIULD_TYPE),
    ('TypeRooms', ROOMS_TYPE),
    ('CeilKind', CEIL_KIND),]
#    ('BldKind', BIULD_KIND)
#                  ]

In [7]:
BOOL_DETAILS = []
#    'Parking', 'Garage']

In [129]:
FEATURES = DETAILS + sum([i[1] for i in NOMINAL_DETAILS],[]) + BOOL_DETAILS

In [9]:
def nominal_vector(name, VALUES):
    vec = [0]*len(VALUES)
    try:
        vec[VALUES.index(name)] = 1
    except:
        pass
    return vec

def yes_or_no(ans):
    if ans.encode('utf-8') == 'да':
        return 1
    if ans.encode('utf-8') == 'Есть':
        return 1
    return 0

In [33]:
def extract_data(flat):
    X = []
    for d in DETAILS:
        try:
            X += [float(flat[d])]
        except:
            X += [0]
    for n in NOMINAL_DETAILS:
        try:
            X += nominal_vector(flat[n[0]],n[1])
        except KeyError:
            X += [0] * len(n[1])
    for b in BOOL_DETAILS:
        try:
            X += [yes_or_no(flat[b])]
        except:
            X += [0]    
    y=0
    try:
        y = float(flat['Price'])
    except:
        print "price problem"
        print flat
    try:
        if flat['PriceCurrencyID'] == '4':
            y *= USD
        if flat['PriceCurrencyID'] == '5':
            y *= EUR
    except:
        pass
    return [X, y]

In [11]:
def extract_data_metrovka(flat):
    X = []
    for d in DETAILS_METROVKA:
        try:
            X += [float(flat[d])]
            if d == 'live_square':
                if X[-1] > X[0]:
                    X[-1] = 0
            if d == 'kitchen_square':
                if X[-1] > X[0]:
                    X[-1] = 0
            if d == 'rooms':
                if X[-1] > 10:
                    X[-1] = 0
            if d == 'square':
                if X[-1] > 900:
                    X[-1] = 0
        except:
            X += [0]
    y=0
    try:
        y = float(flat['price'])
    except:
        print flat['code']
    try:
        if flat['currency'].lower().encode() == 'usd':
            y *= USD
        if flat['currency'].lower().encode() == 'eur':
            y *= EUR
    except:
        pass
    return [X, y]

In [12]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = []):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = 0.18, random_state = 3)
    regr = linear_model.LinearRegression(normalize=True)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [13]:
#remove data without price
#field - str, data - list of dicts
def remove_with_no(field, t_data):
    c = 1
    while c!=0:
        c=0
        for d in t_data:
            try:
                d[field]
            except:
                #print d
                c+=1
                t_data.remove(d)
        print c
    return t_data

In [15]:
def get_X_y_from(t_data):
    t_cleaned_data = [extract_data(i) for i in t_data]
    X = np.array([i[0] for i in t_cleaned_data])
    y = np.array([i[1] for i in t_cleaned_data])
    return X,y

In [ ]:
def get_X_y_from_m(t_data):
    try:        
        t_cleaned_data = [extract_data_metrovka(i) for i in t_data]
    except Exception as e:
        print e.message, e.args
    X = np.array([i[0] for i in t_cleaned_data])
    y = np.array([i[1] for i in t_cleaned_data])
    return X,y

## Load data & clean data

In [16]:
tree = ET.parse('./blagovest/Objects.xml')
root = tree.getroot()

In [130]:
blagovest_data_1 = root.findall('./Object/[@TownCode="300000"][@Transaction="Rent"][@ObjectType="Apartment"]')
blagovest_data_2 = root.findall('./Object/[@TownCode="300000"][@Transaction="Sale"][@ObjectType="Apartment"]')
blagovest_data_1 = [b.attrib for b in blagovest_data_1]
blagovest_data_2 = [b.attrib for b in blagovest_data_2]

In [131]:
blagovest_data_1 = remove_with_no('Price', blagovest_data_1)
blagovest_data_1 = remove_with_no('SqrTotal', blagovest_data_1)
blagovest_data_1 = remove_with_no('NRooms', blagovest_data_1)
blagovest_data_1 = remove_with_no('Floor', blagovest_data_1)
blagovest_data_1 = remove_with_no('RegionCode', blagovest_data_1)
blagovest_data_1 = remove_with_no('Massive', blagovest_data_1)

0
64
4
0
0
0
0
31
1
0


In [139]:
blagovest_data_2 = remove_with_no('Price', blagovest_data_2)
blagovest_data_2 = remove_with_no('SqrTotal', blagovest_data_2)
blagovest_data_2 = remove_with_no('NRooms', blagovest_data_2)
blagovest_data_2 = remove_with_no('Floor', blagovest_data_2)
blagovest_data_2 = remove_with_no('RegionCode', blagovest_data_2)
blagovest_data_2 = remove_with_no('Massive', blagovest_data_2)
blagovest_data_2 = remove_with_no('BldType', blagovest_data_2)
blagovest_data_2 = remove_with_no('SqrLive', blagovest_data_2)
blagovest_data_2 = remove_with_no('SqrKitchen', blagovest_data_2)
blagovest_data_2 = remove_with_no('TypeRooms', blagovest_data_2)
blagovest_data_2 = remove_with_no('CeilKind', blagovest_data_2)

0
0
0
0
0
0
0
0
0
0
0


In [ ]:
import json
with open('metrovka_flats.json') as data_file:
#with open('metrovka_flats_metro.json') as data_file:
#with open('metrovka_flats_district.json') as data_file:    
    json_data = json.load(data_file)
print len(json_data)

#for d in json_data:
#    d['district']=[]

#filter city Kiyv
metrovka_data = [i for i in json_data ] #if i['city'].encode('utf-8') in ["Киев","Київ"]]
print len(metrovka_data)
metrovka_data = remove_with_no('price', metrovka_data)
print len(metrovka_data)
metrovka_data_1 = [i for i in metrovka_data if i['type'] == 1]
metrovka_data_2 = [i for i in metrovka_data if i['type'] == 2]

## First models

In [145]:
blagovest_X_2, blagovest_y_2 = get_X_y_from(blagovest_data_2)

In [146]:
blagovest_X_2[0]

array([ 98.7,  16. ,   4. ,  12. ,  55.2,  13.2,   0. ,   0. ,   0. ,
         0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,
         0. ,   0. ,

In [147]:
regr = process_with(blagovest_X_2, blagovest_y_2, info=True)

Total: 717, train: 587, test: 130
Residual sum of squares: 25107951918.26
Train absolute: 108881.34
Test absolute: 132950.97
Absolute to mean: 5.39%
Train variance score: 0.11
Test variance score: -0.18


In [ ]:
metrovka_X_2, metrovka_y_2 = get_X_y_from_m(metrovka_data_2)

In [ ]:
metrovka_X_2[1]

In [ ]:
print("Residual sum of squares: %.2f"% np.mean((regr.predict(metrovka_X_2) - metrovka_y_2) ** 2))
print("Test absolute: %.2f"% np.mean(abs(regr.predict(metrovka_X_2) - metrovka_y_2)))
print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(metrovka_X_2) - metrovka_y_2))/np.mean(metrovka_y_2)*100))
print('Test variance score: %.2f' % regr.score(metrovka_X_2, metrovka_y_2))

In [ ]:
print "Features sorted by their score:"
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
#f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])

In [ ]:
#!-----extract frequency------
counts = [0] * len(FIELDS)
for d in blagovest_data:
    for k in d.keys():
        try:
            counts[FIELDS.index(k)]+=1
        except:
            pass

together = []
for i in range(0, len(FIELDS)):
    together += [[counts[i], FIELDS[i]]]

together.sort(reverse=True)

for i in range(0, len(FIELDS)):
    together[i] += [round(together[i][0]/float(len(blagovest_data))*100,2)]

for i in together:
    print i
#!---------------------------

## Plots

In [149]:
#show dots from all data
#plt.scatter([np.linalg.norm(i) for i in blagovest_X_2], blagovest_y_2, color='black')
#plt.scatter([np.linalg.norm(i) for i in metrovka_X_2], metrovka_y_2, color='blue')
plt.scatter([np.linalg.norm(i) for i in blagovest_X_2], blagovest_y_2, color='black')
#plt.plot([np.linalg.norm(i) for i in X_2], regr.predict(X_2), color='blue', linewidth=1)
#plt.scatter(np.linalg.norm(X_2[ind]), y_2[ind], color='blue')
#plt.scatter([np.linalg.norm(i) for i in X_2[inx]], y_2[inx], color='blue')
plt.xticks(())
plt.yticks(())
plt.show()

##  Good vs bad

In [ ]:
good = []
bad = []
all = zip(metrovka_X_2, metrovka_y_2)
for i,z in enumerate(all):
    r = regr.predict(z[0])
    err_procent = np.mean(abs(r - z[1])/z[1]*100)
    if err_procent > 30:
        bad += [[metrovka_data_2[i], r[0], err_procent]]
    else:
        good += [[metrovka_data_2[i], r[0], err_procent]]

In [ ]:
print "Всего элементов: %d" % len(metrovka_data_2)
print "Плохих: %d \nХороших: %d" % (len(bad), len(good))
print "Процент плохих: %.2f \nПроцент хороших: %.2f" % (len(bad)/float(len(metrovka_data_2)), len(good)/float(len(metrovka_data_2)))
print "Среднее плохих: %.3f \nСреднее хороших: %.3f" % (np.mean(map(lambda x: x[2], bad)), np.mean(map(lambda x: x[2], good)))
print "Медиана плохих: %.3f \nМедиана хороших: %.3f" % (np.median(map(lambda x: x[2], bad)), np.median(map(lambda x: x[2], good)))

## Manual checking

In [ ]:
ind = 1129
#for i in metrovka_data_2[ind].values():
#    print i
#print metrovka_data_2[ind]
[x,y] = extract_data(metrovka_data_2[ind])
print metrovka_data_2[ind]['rooms']
print
print("%.2f \t diff"% np.mean((regr.predict(x) - y)))
print y, "real"
print regr.predict(x)[0]
print("%.2f"% np.mean(abs(regr.predict(x) - y)/y*100))

In [ ]:
[x,y] =extract_data({
 u'district': u'Оболонский',
 u'price': u'3000000.00',
 #u'rooms': u'2',
 u'square': u'80',
 u'type': 2,
})
print
print("%.2f \t diff"% np.mean((regr.predict(x) - y)))
print y, "real"
print regr.predict(x)[0]
print("%.2f"% np.mean(abs(regr.predict(x) - y)/y*100))

In [ ]:
for d in DISTRICT_NAMES_RU:
    [x,y] =extract_data({
     u'district': d.decode('utf-8'),
     u'price': u'3000000.00',
     #u'rooms': u'2',
     u'square': u'45',
     u'type': 2,
    })
    print "%s: \t %f" %(d,regr.predict(x)[0])

In [ ]:
for d in METRO_RU:
    [x,y] =extract_data({
     u'metro': [d],
     u'price': u'3000000.00',
     #u'rooms': u'2',
     u'square': u'45',
     u'type': 2,
    })
    print "%50s: \t %f" %(d,regr.predict(x)[0])

In [ ]:
from collections import Counter
data = Counter([i['square'] for i in metrovka_data])
data.most_common()   # Returns all unique items and their counts
data.most_common(2)

In [ ]:
len(ADMIN_REGION)
blagovest_X_2[0][4:15]

## Making some fields requiered

In [ ]:
#remove_data_with_no_district_in_X
#[0:4], [4:15]
def make_field_required(t_data):
    ind = []
    X = get_X_y_from(t_data)[0]
    print "Elements: %d" % len(X)
    for i, x in enumerate(X):
        if np.count_nonzero(x[:])<6:
            ind += [i]
    print "Deleted: %d" % len(ind)
    ind = sorted(ind, reverse=True)
    for i in ind:
        t=t_data.pop(i)
    return t_data

In [ ]:
blagovest_data_2=make_field_required(blagovest_data_2)

In [ ]:
metrovka_data_2=make_field_required(metrovka_data_2)

## Delete data with big diffs

In [99]:
def mean_diff(t_data, normalize = True, plots = True, info=True):
    X, y = get_X_y_from(t_data)
    print "Before:"
    before = process_with(X, y, return_short=True)
    print before
    
    if normalize:
        min_max_scaler = preprocessing.MinMaxScaler()
        X = min_max_scaler.fit_transform(X)
        y = min_max_scaler.fit_transform(y)
    
    pts = np.float32(zip(map(lambda x: np.linalg.norm(x), X),y))
    
    mean_pt = np.mean(pts,axis=0)
    mean_x,mean_y = mean_pt
    diffs = np.apply_along_axis(np.linalg.norm,1,pts - mean_pt)
    mean_diff = np.mean(diffs)

    filtered = np.float32([pts[i] for i,diff in enumerate(diffs) if diff <= mean_diff])
    filtered_mean = np.mean(filtered,axis=0)
    
    t_data_f = [t_data[i] for i,diff in enumerate(diffs) if diff <= mean_diff]
    X_f, y_f = get_X_y_from(t_data_f)
    print "After:"
    after = process_with(X_f,y_f, return_short=True)
    print after
    
#    plt.plot(pts[:,0],pts[:,1],'ro')
#    plt.plot(mean_x,mean_y,'w^')
#    plt.plot(filtered[:,0],filtered[:,1],'bo')
#    plt.plot(filtered_mean[0],filtered_mean[1],'w^')
#    plt.show()
    if after[1] > before[1] or after[0] < before[0]:
        return t_data_f
    return t_data

## Delete "outlaws"

In [25]:
def get_ind_by_y(X, y, ind_max = True):
    if ind_max:
        return np.argmax(y)
    else:
        return np.argmin(y)

In [26]:
def get_ind_by_x_norm(X, y, ind_max=True):
    norms = [np.linalg.norm(i) for i in X]
    #sorted_norm = [np.linalg.norm(i) for i in X]
    #sorted_norm.sort()
    if ind_max:
        #ind = norms.index(sorted_norm[-1])
        ind = norms.index(max(norms))
    else:
        #ind = norms.index(sorted_norm[1])
        ind = norms.index(min(norms))
    return ind

In [27]:
def get_ind_by_y_to_x_norm(X, y, ind_max=True):
    y_to_norm_x = [y[i]/np.linalg.norm(X[i]) for i in range(0,len(X))]
    if ind_max:
        return np.argmax(y_to_norm_x)
    else:
        return np.argmin(y_to_norm_x)

In [120]:
def cut_data_by(t_data, get_ind_cut_function, ind_max_flag=True):
    previous_sum = 10**32
    previous_score = -10**32
    X, y = get_X_y_from(t_data)
    [current_sum, current_score] = process_with(X, y, return_short=True)
    temp={}
    while current_score > previous_score or current_sum < previous_sum:
        previous_score = current_score
        previous_sum = current_sum

        inx = get_ind_cut_function(X, y,ind_max=ind_max_flag)
        #print t_data[norms.index(sorted_norm[-1])]['code'], sorted_norm[-1]
        temp = t_data.pop(inx)
        
        X, y = get_X_y_from(t_data)
        [current_sum, current_score] = process_with(X, y, return_short=True)
        
        print current_sum, previous_sum
        print current_score, previous_score
        print "-----"
    if temp != {}:
        t_data+=[temp]
    return t_data

## One by one

In [144]:
l = 1
while l!=len(blagovest_data_2):
    l = len(blagovest_data_2)
    print "by x norm max"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_x_norm, True)
    print "by y max"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_y, True)
    print "by y to x norm max"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_y_to_x_norm, True)
    print "by x norm min"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_x_norm, False)
    print "by y min"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_y, False)
    print "by y to x norm min"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_y_to_x_norm, False)
    print "mean_diff"
    blagovest_data_2 = mean_diff(blagovest_data_2, normalize = True)

by x norm max
2593410.42731 1.80735840999e+15
0.602896601249 -5.81250645275e+19
-----
9.8151742527e+16 2593410.42731
-2.26499041109e+23 0.602896601249
-----
by y max
1.33407601992e+18 2628203.34132
-1.69457405797e+25 0.613235217489
-----
by y to x norm max
1.53085075588e+17 9.00738016087e+16
-4.4582315964e+23 -1.71351944461e+23
-----
by x norm min
9.36594005405e+17 2.07453783234e+17
-1.29522917545e+25 -5.92714958217e+23
-----
by y min
2.00437242617e+17 1.32027762709e+18
-9.11228396137e+23 -4.1355667738e+25
-----
3.13643880928e+17 2.00437242617e+17
-1.03961678283e+24 -9.11228396137e+23
-----
by y to x norm min
2594962.49078 8.08333637586e+17
0.554583899078 -7.05266219457e+24
-----
2658501.30438 2594962.49078
0.589669756496 0.554583899078
-----
2544942.29539 2658501.30438
0.591149127403 0.589669756496
-----
2.94146963814e+15 2544942.29539
-2.23977499948e+20 0.591149127403
-----
mean_diff
Before:
(979263990083984.62, -2.7027818592308007e+19)


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


After:
(1.8839073473722349e+17, -4.7746426812681446e+24)
by x norm max
1.99746694926e+16 9.79263990084e+14
-1.0328443012e+22 -2.70278185923e+19
-----
by y max
8.69796910306e+15 3.95505682712e+16
-1.95844497763e+21 -4.40876432583e+22
-----
1.96516644551e+16 8.69796910306e+15
-2.48067695563e+21 -1.95844497763e+21
-----
by y to x norm max
1.98445603353e+16 5.40378456701e+17
-2.5296153899e+21 -7.3644151705e+24
-----
1.51218737751e+16 1.98445603353e+16
-6.1205573119e+21 -2.5296153899e+21
-----
1.11513558941e+15 1.51218737751e+16
-3.35969134175e+19 -6.1205573119e+21
-----
2527011.87447 1.11513558941e+15
0.572803823795 -3.35969134175e+19
-----
2691494.40233 2527011.87447
0.557538507767 0.572803823795
-----
by x norm min
2854972.6516 2849949.25242
0.530218178181 0.554994060114
-----
by y min
2640344.85374 2701998.51821
0.562276509633 0.603083488576
-----
2628621.82651 2640344.85374
0.588116529077 0.562276509633
-----
2603866.75911 2628621.82651
0.591155121223 0.588116529077
-----
1.47483949007

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


After:
(2.0317307465138668e+18, -2.7461940505340418e+26)
by x norm max
1.65523593903e+18 1.24546917425e+18
-1.59320115585e+25 -2.05429918075e+25
-----
8.42948281566e+17 1.65523593903e+18
-1.62516653957e+25 -1.59320115585e+25
-----
1.1287287026e+18 8.42948281566e+17
-3.62271707732e+25 -1.62516653957e+25
-----
by y max
1.02010450593e+18 1.44956856629e+18
-2.95899759591e+25 -5.33406756697e+25
-----
2.2280210159e+17 1.02010450593e+18
-1.2884086117e+24 -2.95899759591e+25
-----
3.35755849509e+17 2.2280210159e+17
-3.00113001683e+24 -1.2884086117e+24
-----
by y to x norm max
3.35755849509e+17 2.414760952e+17
-3.00113001683e+24 -1.51343349639e+24
-----
by x norm min
2.34052157571e+17 2.414760952e+17
-1.45835391958e+24 -1.51343349639e+24
-----
6.25780784208e+17 2.34052157571e+17
-1.25544809462e+25 -1.45835391958e+24
-----
by y min
4.26632391736e+17 6.66942898271e+17
-5.151304823e+24 -1.3731111155e+25
-----
4.07393783357e+17 4.26632391736e+17
-1.89274731595e+24 -5.151304823e+24
-----
2.6555924389

...

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


9.47560872697e+16 5.25227186659e+15
-2.79751242187e+25 -9.88057366818e+22
-----
by y max
7.67727330811e+16 1.97046119995e+17
-1.87798867509e+25 -1.39066825236e+26
-----
1.90733229899e+17 7.67727330811e+16
-1.34786177263e+26 -1.87798867509e+25
-----
by y to x norm max
1.96426519579e+17 133081.307692
-1.29039034793e+26 -0.251325137243
-----
by x norm min
4.23267796834e+17 132392.630769
-1.16743104626e+27 -0.263103841634
-----
by y min
1.31658474246e+16 123884.107692
-1.16269981236e+24 -0.123789486655
-----
by y to x norm min
1.67456351502e+16 133744.707692
-1.88093252363e+24 -0.175892492859
-----
mean_diff
Before:
(129572.03076923077, -0.16535216270853659)
After:
(4.2275242271411616e+17, -1.3289665443350483e+27)
by x norm max


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


1.23742309806e+16 129572.030769
-1.02708503982e+24 -0.165352162709
-----
by y max
6.32257224184e+16 135582.907692
-2.68137507023e+25 -0.220396137155
-----
by y to x norm max
7.61229144336e+16 135705.092308
-3.88687685322e+25 -0.189458524505
-----
by x norm min
4.14061523842e+16 128729.984615
-1.15000497421e+25 -0.0973446276049
-----
by y min
3.98026133205e+16 128905.646154
-1.0626570838e+25 -0.149184895927
-----
by y to x norm min
4.12226903234e+16 125663.615385
-1.13983668467e+25 -0.0935218274822
-----
mean_diff
Before:
(132950.96923076923, -0.18112020376763227)
After:
(4.2275242271411616e+17, -1.3289665443350483e+27)


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


## Saving and loading coefs

In [ ]:
coefs = map(lambda x: round(x, 4), regr.coef_)
f = zip(coefs, FEATURES)
print coefs

In [ ]:
import pickle
with open('filtered_mixed_coefs_district_square_required.json', 'w') as outfile:
    pickle.dump(coefs, outfile)

In [ ]:
import pickle
with open('filtered_mixed_coefs_district_square_required.json', 'r') as infile:
    c = pickle.load(infile)

## Save data

In [ ]:
with open('metrovka_flats_metro.json', 'w') as outfile:
    json.dump(metrovka_data, outfile)
#json.dumps(data_2[0:1])